# DataFrames feature 


## Creation d'un dataframe avec pandas

Nous allons commencer par créer un dataframe `product_data` définie à partir des données 
```python
product_data = {
    'product a': [13, 20, 0, 10], 
    'product b': [10, 30, 17, 20],
    'product c': [6, 9, 10, 0]
}```


In [ ]:
import pandas as pd

In [ ]:
# données sous la forme de dictionnaire

product_data = {
    'product a': [13, 20, 0, 10], 
    'product b': [10, 30, 17, 20],
    'product c': [6, 9, 10, 0]
}

In [ ]:
product_data

### Transformation du dictionnair en data frame


In [ ]:
purchase_data = pd.DataFrame(product_data)

In [ ]:
purchase_data.head()

### Ajouter l'index

In [ ]:

purchase_data = pd.DataFrame(product_data, 
                             index=['Ronny', 'Bobby', 'Ricky', 'Mike'])

In [ ]:
purchase_data.head()

### Opérations sur les dataframes

* Filtrage

In [ ]:
purchase_data.loc['Ricky'] >0

In [ ]:
# d'une manière équivalente 
purchase_data.iloc[2] >0

In [ ]:
purchase_data.iloc[0] >10

In [ ]:
purchase_data.iloc[:,0]

In [ ]:
purchase_data.iloc[0]

### Dataframe à partir d'un fichier csv

In [ ]:
df = pd.read_csv('../data/sources/purshases.csv', index_col=0)

df


Excuter la cellule précédente en enlevant ```index_col=0```

### DataFrame à partir d'un fichier json

In [ ]:
df = pd.read_json('../data/sources/purshases.json')

df

In [ ]:
DF=df.transpose()
DF

In [ ]:
df.transpose().to_json('../data/sources/purshases_t.json')
df

### Base de donné à partir d'un dataframe

> Nous allons créer un fichier de base de donnée à partir du data frame **DF**

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>apples</th>      <th>oranges</th>    </tr>  </thead>  <tbody>    <tr>      <th>David</th>      <td>1</td>      <td>2</td>    </tr>    <tr>      <th>June</th>      <td>3</td>      <td>0</td>    </tr>    <tr>      <th>Lilly</th>      <td>0</td>      <td>7</td>    </tr>    <tr>      <th>Robert</th>      <td>2</td>      <td>3</td>    </tr>  </tbody></table>'


#### Création du contexte

In [ ]:
import sqlite3
connection = sqlite3.connect('db.sqlite')
cursor = connection.cursor()
cursor.execute('Create Table if not exists purshases (id Text, apples Integer, oranges Integer)')
connection.commit()


#### Ingestion du dataframe Df dans la base de donnée

In [ ]:
DF.to_sql('purshases', connection, if_exists='replace', index = True)
connection.close()

In [ ]:
con2=sqlite3.connect('db.sqlite')

In [ ]:
df2 = pd.read_sql_query("SELECT * FROM purshases", con2)

df2

##### Remarques 

* Pour l'installation des bibliothèques utilisé dans ce notebook :
    * ``` !pip install pysqlite3 ```
    * ``` !pip install <nom du package> ```

### Importer une database en dataframe

In [ ]:
con = sqlite3.connect("../data/sources/customer_sales.db")

* Explorer les tablesde la base de donnée 

In [ ]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
df = pd.read_sql_query("SELECT * FROM tbl_products", con)

df

In [ ]:
df = df.set_index('Product_ID')

df

* Pour enrejistrer un dataframe en format ```csv```

In [ ]:
df.to_csv('new_purchases.csv')

## From CSV to DataFrame

- Dans cette partie nous allons:
    * Explorer le fichier CSV **`IMDB-Movie-Data.csv`**
    * Voir l'utilité des méthodes `.head()` , `tail()`,`.info()`, `.shape`, `.append()`, ...
    * Appliquer quelques transformations au dataframe
    * Calculer quelques informations statistiques
 
 
 Essayer de décrire l'utilité de chaque opération et transformation :



In [ ]:
movies_df = pd.read_csv("../data/sources/IMDB-Movie-Data.txt", index_col="Title")

In [ ]:
movies_df.head()

In [ ]:
movies_df.tail(2)

In [ ]:
movies_df.info()

In [ ]:
movies_df.shape

In [ ]:
temp_df = movies_df.append(movies_df)

temp_df.shape
temp_df

In [ ]:
temp_df = temp_df.drop_duplicates()

temp_df.shape

In [ ]:
temp_df.drop_duplicates(inplace=True)
temp_df.shape

In [ ]:
temp_df = movies_df.append(movies_df)  # make a new copy

temp_df.drop_duplicates(inplace=True, keep=False)

temp_df.shape

In [ ]:
movies_df.columns

In [ ]:
movies_df.rename(columns={
        'Runtime (Minutes)': 'Runtime', 
        'Revenue (Millions)': 'Revenue_millions'
    }, inplace=True)


movies_df.columns

In [ ]:
movies_df.columns = ['Rank', 'genre', 'description', 'director', 'actors', 'year', 'runtime', 
                     'rating', 'votes', 'revenue_millions', 'metascore']


movies_df.columns

In [ ]:
movies_df.columns = [col.lower() for col in movies_df]

movies_df.columns

In [ ]:
movies_df.isnull()

In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.dropna()

In [ ]:
movies_df.dropna(axis=1)

In [ ]:
revenue = movies_df['revenue_millions']
revenue

In [ ]:
revenue.head()

In [188]:
revenue_mean = revenue.mean()

revenue_mean

82.95637614678898

In [189]:
revenue.fillna(revenue_mean, inplace=True)

In [190]:
movies_df.isnull().sum()

rank                 0
genre                0
description          0
director             0
actors               0
year                 0
runtime              0
rating               0
votes                0
revenue_millions     0
metascore           64
dtype: int64

In [191]:
movies_df.describe()

,rank,year,runtime,rating,votes,revenue_millions,metascore
count,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,936.000000
mean,500.500000,2012.783000,113.172000,6.723200,1.698083e+05,82.956376,58.985043
std,288.819436,3.205962,18.810908,0.945429,1.887626e+05,96.412043,17.194757
min,1.000000,2006.000000,66.000000,1.900000,6.100000e+01,0.000000,11.000000
25%,250.750000,2010.000000,100.000000,6.200000,3.630900e+04,17.442500,47.000000
50%,500.500000,2014.000000,111.000000,6.800000,1.107990e+05,60.375000,59.500000
75%,750.250000,2016.000000,123.000000,7.400000,2.399098e+05,99.177500,72.000000
max,1000.000000,2016.000000,191.000000,9.000000,1.791916e+06,936.630000,100.000000


In [192]:
movies_df.sort_values(by='rank')

,rank,genre,description,director,actors,year,runtime,rating,votes,revenue_millions,metascore
Title,,,,,,,,,,,
Guardians of the Galaxy,1,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.130000,76.0
Prometheus,2,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.460000,65.0
Split,3,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.120000,62.0
Sing,4,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.320000,59.0
Suicide Squad,5,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.020000,40.0
...,...,...,...,...,...,...,...,...,...,...,...
Secret in Their Eyes,996,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,82.956376,45.0
Hostel: Part II,997,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.540000,46.0
Step Up 2: The Streets,998,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.010000,50.0


In [193]:
movies_df.sort_index(1)

C:\Users\User\AppData\Local\Temp\ipykernel_4556\1758315711.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  movies_df.sort_index(1)


,actors,description,director,genre,metascore,rank,rating,revenue_millions,runtime,votes,year
Title,,,,,,,,,,,
Guardians of the Galaxy,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",A group of intergalactic criminals are forced ...,James Gunn,"Action,Adventure,Sci-Fi",76.0,1,8.1,333.130000,121,757074,2014
Prometheus,"Noomi Rapace, Logan Marshall-Green, Michael Fa...","Following clues to the origin of mankind, a te...",Ridley Scott,"Adventure,Mystery,Sci-Fi",65.0,2,7.0,126.460000,124,485820,2012
Split,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"Horror,Thriller",62.0,3,7.3,138.120000,117,157606,2016
Sing,"Matthew McConaughey,Reese Witherspoon, Seth Ma...","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Animation,Comedy,Family",59.0,4,7.2,270.320000,108,60545,2016
Suicide Squad,"Will Smith, Jared Leto, Margot Robbie, Viola D...",A secret government agency recruits some of th...,David Ayer,"Action,Adventure,Fantasy",40.0,5,6.2,325.020000,123,393727,2016
...,...,...,...,...,...,...,...,...,...,...,...
Secret in Their Eyes,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...","A tight-knit team of rising investigators, alo...",Billy Ray,"Crime,Drama,Mystery",45.0,996,6.2,82.956376,111,27585,2015
Hostel: Part II,"Lauren German, Heather Matarazzo, Bijou Philli...",Three American college students studying abroa...,Eli Roth,Horror,46.0,997,5.5,17.540000,94,73152,2007
Step Up 2: The Streets,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",Romantic sparks occur between two dance studen...,Jon M. Chu,"Drama,Music,Romance",50.0,998,6.2,58.010000,98,70699,2008


## pandas method
##### Transformation du dictionnaire en data frame
* `DataFrame(product_data)` : transformer le dictionnaire en Data Frame (tableau) 
* `df.head(n)` : afficher les n premiéres lignes
    * `df.head()` : afficher par defaut les $5$ premiéres lignes
* `df.tail(n)` :afficher les n derniéres lignes
##### Ajouter l'index
* `df.DataFrame(product_data,index=['Ronny', 'Bobby', 'Ricky', 'Mike'])` : ajouter les indexes
#####  Opérations sur les dataframes
* `DataFrame.loc['Ricky'] >12` : chercher les colonnes de la ligne d'index 'Ricky' qui ont un entier > à 12 type de retour: tableau booléen
    * `DataFrame.iloc[i] `: accéder à la ligne d'index i (position) 
    * `purchase_data.iloc[:,0]` : accéder à la colonne d'index 0
##### Dataframe à partir d'un fichier csv
* `pd.read_csv('../data/sources/purshases.csv', index_col=0)` : lire le fichier .csv à partir du chemin en paramétre en attribuant l'indice les valeurs de la colonne d'indice 0
##### DataFrame à partir d'un fichier json
* `pd.read_json('../data/sources/purshases.json')` : lire un fichier json
* `DF=df.transpose()` : les nouvelles lignes seront les colonnes et les  nouvelles colonnes seront les lignes
    * `df.transpose().to_json('../data/sources/purshases_t.json')`: transposer les lignes et les colonnes à partir d'un fichier json 
##### Base de donné à partir d'un dataframe
* `DF.to_sql('purshases', connection, if_exists='replace', index = True)`: Écrire des enregistrements stockés dans un DataFrame dans une base de données SQL. 
    * `purchases` : nom de la table sql
    * `connection` : la base de connexion
    * `if_exists='replace` : si la table existe déjà la remplacer,supprimez la table avant d'insérer de nouvelles valeurs.
    * `index = True` : écrire l'index DataFrame sous forme de colonne.
* `pd.read_sql_query("SELECT * FROM purshases", con2)` : lire la requête SQL dans un DataFrame.
    * `SELECT * FROM purshases` : requête SQL à exécuter
    * `con2` : base de donnée de connexion
##### Importer une database en dataframe
* `con = sqlite3.connect("../data/sources/customer_sales.db")` :  importer une database en dataframe
* `df = df.set_index('Product_ID')` : met l'index  du DataFrame à l'aide de colonnes existantes.
* `df.to_csv('new_purchases.csv')`: pour enregistrer un dataframe en format ```csv```
##### From CSV to DataFrame
* `movies_df = pd.read_csv("../data/sources/IMDB-Movie-Data.txt", index_col="Title")` : lire un fichier csv dans DataFrame en attribuant l'indice les valeurs de la colonne  Title.
* `movies_df.info()` : Cette méthode retourne des informations sur un DataFrame
* `movies_df.shape()` :Renvoie un tuple représentant la dimensionnalité du DataFrame.
* `temp_df = temp_df.drop_duplicates()` : renvoie DataFrame et supprime les lignes en double.
    * `temp_df.drop_duplicates(inplace=True)`
* `movies_df.columns` : renvoie un tableau contient les noms des colonnes  du Dataframe
* `movies_df.rename(columns={'Runtime (Minutes)': 'Runtime'}, inplace=True)` : renommer la colonne
    * `movies_df.columns = [col.lower() for col in movies_df]` : met la colonne en miniscules
* `movies_df.isnull()` : renvoie un objet booléen  indiquant si les valeurs sont nuls.
    * `movies_df.isnull().sum()` : renvoie un tableau avec la somme des valeurs nuls pour chaque colonne.
* `movies_df.dropna()` : Supprimer les valeurs manquantes.
    * `movies_df.dropna(axis=1)` : (1 ou columns) supprimer les colonnes qui contiennent une valeur manquante.
* `revenue = movies_df['revenue_millions']` : renvoie les lignes de la colonne en paramétre
* `revenue.mean()` : Renvoie la moyenne des valeurs de la colonne revenue.
* `revenue.fillna(revenue_mean, inplace=True)` :Remplir les valeurs non nuls à l'aide de la méthode mean().
    * `inplace=True`: remplissez sur place
* `movies_df.describe()` :générer des statistiques descriptives.
* `movies_df.sort_values(by='rank')` : Triez par les valeurs par la colonne demandé.
* `movies_df.sort_index(1)` : Triez par les valeurs selon l'axe d'indice 1 = les colonnes




 






















 



In [ ]:
#https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/